## Check the ground truth active and reactive residuals in the data
- Useful to compute active and reactive residuals of the solvers

In [ ]:
from gridFM.datasets.powergrid import GridDatasetMem
from gridFM.datasets.data_normalization import BaseMVANormalizer
from gridFM.utils.loss import PBELoss
from gridFM.datasets.transforms import AddRandomMask

import torch
from torch_geometric.loader import DataLoader
from tqdm import tqdm
import os


torch.set_printoptions(precision=10)
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(f"Using device: {device}")

In [ ]:
base_path = os.path.join(os.getcwd(), "../data/")
network = "case30_ieee_ac"

In [ ]:
data_path = os.path.join(base_path, network)
scenarios = 512
norm_method = "baseMVAnorm"
node_normalizer, edge_normalizer = (
    BaseMVANormalizer(node_data=True),
    BaseMVANormalizer(node_data=False),
)
mask_ratio = 0.5
mask_dim = 6
mask_value = 0
pe_dim = 20

In [ ]:
dataset = GridDatasetMem(
    root=data_path,
    norm_method=norm_method,
    node_normalizer=node_normalizer,
    edge_normalizer=edge_normalizer,
    pe_dim=pe_dim,
    mask_dim=mask_dim,
    transform=AddRandomMask(mask_dim=mask_dim, mask_ratio=mask_ratio),
)

In [ ]:
loader = DataLoader(dataset, batch_size=16, shuffle=False)

In [ ]:
loss = PBELoss()

In [ ]:
loss_sum = 0.0
power_loss_sum = 0.0
active_power_loss_sum = 0.0
reactive_power_loss_sum = 0.0
num_batches = 0

# Loop through the batches
for batch in tqdm(loader):
    metric = loss(batch.y, batch.y, batch.edge_index, batch.edge_attr, batch.mask)

    # Accumulate the metrics
    loss_sum += metric["loss"].item()
    power_loss_sum += metric["Power power loss in p.u."]
    active_power_loss_sum += metric["Active Power Loss in p.u."]
    reactive_power_loss_sum += metric["Reactive Power Loss in p.u."]
    num_batches += 1

# Calculate the means
mean_loss = loss_sum / num_batches
mean_power_loss = power_loss_sum / num_batches
mean_active_power_loss = active_power_loss_sum / num_batches
mean_reactive_power_loss = reactive_power_loss_sum / num_batches

In [ ]:
print("BaseMVA (MVA): ", node_normalizer.baseMVA.numpy())

In [ ]:
print(f"Mean Power Power Loss in MVA: {mean_power_loss * node_normalizer.baseMVA}")
print(
    f"Mean Active Power Loss in MW: {mean_active_power_loss * node_normalizer.baseMVA}"
)
print(
    f"Mean Reactive Power Loss in MVar: {mean_reactive_power_loss * node_normalizer.baseMVA}"
)